In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from pyomo.opt import SolverFactory
from pyomo.environ import *
import pyomo.environ as pyo
import pyomo.gdp as gdp

In [2]:
import shutil
import sys
import os.path
if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

/bin/sh: conda: command not found


## Prepare dataset

In [3]:
df = pd.read_csv('Restaurants.csv')

In [4]:
df['address'] = df['address'] + ',Los Angeles,CA'

In [5]:
df.head()

,name,rating,reviewCount,phone,priceRange,address,neighborhoods,url
0,Howlin’ Ray’s,4.5,6639,(213) 935-8399,$$,"727 N Broadway,Los Angeles,CA",Chinatown,https://www.yelp.com/biz/howlin-rays-los-angel...
1,Wurstküche,4.0,8448,(213) 687-4444,$$,"800 E 3rd St,Los Angeles,CA",Arts District,https://www.yelp.com/biz/wurstk%C3%BCche-los-a...
2,Daikokuya Little Tokyo,4.0,8645,(213) 626-1680,$$,"327 E 1st St,Los Angeles,CA",Little Tokyo,https://www.yelp.com/biz/daikokuya-little-toky...
3,Slurpin’ Ramen Bar - Los Angeles,4.5,4640,(213) 388-8607,$$,"3500 W 8th St,Los Angeles,CA",Koreatown,https://www.yelp.com/biz/slurpin-ramen-bar-los...
4,Morrison Atwater Village,4.5,4461,(323) 667-1839,$$,"3179 Los Feliz Blvd,Los Angeles,CA",Atwater Village,https://www.yelp.com/biz/morrison-atwater-vill...


In [6]:
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

locator = Nominatim(user_agent='myGeocoder')

# 1 - conveneint function to delay between geocoding calls
#geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['address'].apply(locator.geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [7]:
df = df.drop(columns = ['point','altitude'])

In [8]:
df.head()

,name,rating,reviewCount,phone,priceRange,address,neighborhoods,url,location,latitude,longitude
0,Howlin’ Ray’s,4.5,6639,(213) 935-8399,$$,"727 N Broadway,Los Angeles,CA",Chinatown,https://www.yelp.com/biz/howlin-rays-los-angel...,"(LASA, 727, North Broadway, New Chinatown, Chi...",34.061519,-118.239473
1,Wurstküche,4.0,8448,(213) 687-4444,$$,"800 E 3rd St,Los Angeles,CA",Arts District,https://www.yelp.com/biz/wurstk%C3%BCche-los-a...,"(800, East 3rd Street, Long Beach, Los Angeles...",33.770423,-118.182243
2,Daikokuya Little Tokyo,4.0,8645,(213) 626-1680,$$,"327 E 1st St,Los Angeles,CA",Little Tokyo,https://www.yelp.com/biz/daikokuya-little-toky...,"(Daikokuya, 327, East 1st Street, Little Tokyo...",34.049971,-118.240083
3,Slurpin’ Ramen Bar - Los Angeles,4.5,4640,(213) 388-8607,$$,"3500 W 8th St,Los Angeles,CA",Koreatown,https://www.yelp.com/biz/slurpin-ramen-bar-los...,"(3500, West 8th Street, Koreatown, Los Angeles...",34.057590,-118.306725
4,Morrison Atwater Village,4.5,4461,(323) 667-1839,$$,"3179 Los Feliz Blvd,Los Angeles,CA",Atwater Village,https://www.yelp.com/biz/morrison-atwater-vill...,"(The Morrison, 3179, Los Feliz Boulevard, Atwa...",34.123650,-118.268699


In [9]:
guanda = '1471 W 37th St, Los Angeles, CA'
guanda = locator.geocode(guanda).point[:2]
sida = '900 W Temple St, Los Angeles, CA'
sida = locator.geocode(sida).point[:2]

In [10]:
xinyao = '1279 w 37th pl, Los Angeles, CA'
xinyao = locator.geocode(xinyao).point[:2]
feifan = '1221 w 3rd st, Los Angeles, CA'
feifan = locator.geocode(feifan).point[:2]

In [11]:
# Distance calculation formula given lat & lng:
# 69.5 * abs(lat1 - lat2) + 57.3 * abs(lon1 - lon2) miles.
df['distance_guanda'] = 69.5 * abs(df['latitude'] - guanda[0]) + 57.3 * abs(df['longitude'] - guanda[1])

In [12]:
df['distance_sida'] = 69.5 * abs(df['latitude'] - sida[0]) + 57.3 * abs(df['longitude'] - sida[1])

In [13]:
df['distance_xinyao'] = 69.5 * abs(df['latitude'] - xinyao[0]) + 57.3 * abs(df['longitude'] - xinyao[1])

In [14]:
df['distance_feifan'] = 69.5 * abs(df['latitude'] - feifan[0]) + 57.3 * abs(df['longitude'] - feifan[1])

In [15]:
df['max_distance'] = df[['distance_guanda','distance_sida','distance_xinyao','distance_feifan']].max(axis=1)

In [16]:
scaler = MinMaxScaler()
df['reviewCount'] = scaler.fit_transform(df['reviewCount'].values.reshape(-1,1))

In [17]:
df['pricerange'] = df['priceRange'].replace(['$','$$','$$$','$$$$'], [10, 30, 60, 100000])

In [18]:
df.head()

,name,rating,reviewCount,phone,priceRange,address,neighborhoods,url,location,latitude,longitude,distance_guanda,distance_sida,distance_xinyao,distance_feifan,max_distance,pricerange
0,Howlin’ Ray’s,4.5,0.758313,(213) 935-8399,$$,"727 N Broadway,Los Angeles,CA",Chinatown,https://www.yelp.com/biz/howlin-rays-los-angel...,"(LASA, 727, North Broadway, New Chinatown, Chi...",34.061519,-118.239473,6.399578,0.631031,6.175737,1.407428,6.399578,30.0
1,Wurstküche,4.0,0.976265,(213) 687-4444,$$,"800 E 3rd St,Los Angeles,CA",Arts District,https://www.yelp.com/biz/wurstk%C3%BCche-los-a...,"(800, East 3rd Street, Long Beach, Los Angeles...",33.770423,-118.182243,24.284729,24.034025,23.921677,24.358956,24.358956,30.0
2,Daikokuya Little Tokyo,4.0,1.000000,(213) 626-1680,$$,"327 E 1st St,Los Angeles,CA",Little Tokyo,https://www.yelp.com/biz/daikokuya-little-toky...,"(Daikokuya, 327, East 1st Street, Little Tokyo...",34.049971,-118.240083,5.562051,1.291173,5.338210,1.616105,5.562051,30.0
3,Slurpin’ Ramen Bar - Los Angeles,4.5,0.517470,(213) 388-8607,$$,"3500 W 8th St,Los Angeles,CA",Koreatown,https://www.yelp.com/biz/slurpin-ramen-bar-los...,"(3500, West 8th Street, Koreatown, Los Angeles...",34.057590,-118.306725,2.806197,3.495584,3.169248,2.731969,3.495584,30.0
4,Morrison Atwater Village,4.5,0.495904,(323) 667-1839,$$,"3179 Los Feliz Blvd,Los Angeles,CA",Atwater Village,https://www.yelp.com/biz/morrison-atwater-vill...,"(The Morrison, 3179, Los Feliz Boulevard, Atwa...",34.123650,-118.268699,9.043013,5.469148,8.819172,5.144217,9.043013,30.0


In [19]:
df['priceRange'] = df['priceRange'].replace(['$','$$','$$$','$$$$'], [1, 2, 3, 4])

In [20]:
#df.to_csv('inputs.csv')

In [51]:
inputs = pd.read_csv('inputs.csv')

In [52]:
inputs = inputs.dropna()

In [53]:
inputs.reset_index(inplace = True)

In [54]:
inputs

,index,Unnamed: 0,name,rating,reviewCount,phone,priceRange,address,neighborhoods,url,location,latitude,longitude,distance_guanda,distance_sida,distance_xinyao,distance_feifan,max_distance,pricerange
0,0,0,Howlin’ Ray’s,4.5,0.758313,(213) 935-8399,2.0,"727 N Broadway,Los Angeles,CA",Chinatown,https://www.yelp.com/biz/howlin-rays-los-angel...,"Howlin' Ray's, 727, North Broadway, New Chinat...",34.061548,-118.239711,6.387903,0.619357,6.164062,1.395753,6.387903,30.0
1,1,1,Wurstküche,4.0,0.976265,(213) 687-4444,2.0,"800 E 3rd St,Los Angeles,CA",Arts District,https://www.yelp.com/biz/wurstk%C3%BCche-los-a...,"800, 3rd Street, Burbank, Los Angeles County, ...",34.186872,-118.313947,12.205116,12.455820,12.568167,12.130888,12.568167,30.0
2,2,2,Daikokuya Little Tokyo,4.0,1.000000,(213) 626-1680,2.0,"327 E 1st St,Los Angeles,CA",Little Tokyo,https://www.yelp.com/biz/daikokuya-little-toky...,"Daikokuya, 327, East 1st Street, Little Tokyo,...",34.049971,-118.240083,5.562051,1.291173,5.338210,1.616105,5.562051,30.0
3,3,3,Slurpin’ Ramen Bar - Los Angeles,4.5,0.517470,(213) 388-8607,2.0,"3500 W 8th St,Los Angeles,CA",Koreatown,https://www.yelp.com/biz/slurpin-ramen-bar-los...,"West 8th Street, South Park, Downtown, Los Ang...",34.044398,-118.255704,4.279628,1.488919,4.055787,1.108384,4.279628,30.0
4,4,4,Morrison Atwater Village,4.5,0.495904,(323) 667-1839,2.0,"3179 Los Feliz Blvd,Los Angeles,CA",Atwater Village,https://www.yelp.com/biz/morrison-atwater-vill...,"The Morrison, 3179, Los Feliz Boulevard, Atwat...",34.123650,-118.268699,9.043013,5.469148,8.819172,5.144217,9.043013,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,235,235,Sushi Ippo,4.0,0.135181,(213) 381-0110,2.0,"3800 Wilshire Blvd,Los Angeles,CA",Koreatown,https://www.yelp.com/biz/sushi-ippo-los-angele...,"Wilshire Boulevard, Santa Monica, Los Angeles ...",34.040094,-118.472421,11.084579,14.205952,11.447630,13.429556,14.205952,30.0
227,236,236,Doomie’s Home Cookin’,4.0,0.174217,(323) 469-4897,2.0,"1253 Vine St,Los Angeles,CA",Hollywood,https://www.yelp.com/biz/doomies-home-cookin-l...,"Atch-kotch, 1253 #5, Vine Street, Hollywood, L...",34.094043,-118.327832,6.549119,6.799822,6.912170,6.474891,6.912170,30.0
228,237,237,JiST Cafe,4.0,0.096386,(213) 792-2116,2.0,"116 Judge John Aiso St,Los Angeles,CA",Little Tokyo,https://www.yelp.com/biz/jist-cafe-los-angeles...,"JiST Cafe, 116, Judge John Aiso Street, Little...",34.050764,-118.240384,5.599923,1.218818,5.376082,1.543750,5.599923,30.0
229,238,238,Ramen Hood,4.5,0.080120,(213) 285-7571,2.0,"317 S Broadway,Los Angeles,CA",Downtown,https://www.yelp.com/biz/ramen-hood-los-angele...,"Golden Road Grand Central Market, 317, South B...",34.050645,-118.248829,5.107763,0.743227,4.883922,1.068158,5.107763,30.0


In [55]:
weighted_rating = inputs['rating']*inputs['reviewCount']
max_distance = inputs['max_distance']
price_range = inputs['priceRange']

In [56]:
N=len(inputs)

## Set user preference and limitation

In [57]:
# budget_i: budget input of 4 people for each month ------- Question: use average budget or not
budget_1 = [2,2,4,3] 
budget_2 = [3,2,4,3]
budget_3 = [2,3,4,3]
budget_4 = [3,2,3,4]
budget_5 = [2,2,2,3]
budget = [np.mean(budget_1),np.mean(budget_2),np.mean(budget_3),np.mean(budget_4),np.mean(budget_5)]


# distance tolerance i: distance tolerance input for each month
distance_tolerance = [5,10,15,20,25]

## Formulate deterministic programming model by Pyomo

In [58]:
# Create Model
m = pyo.ConcreteModel()


m.N = pyo.Set(initialize=range(1, N+1))


# Decision variables describing whether a restaurant is chose
# x_i: Decision variable for i th month
m.x_1 = pyo.Var(m.N, domain = Binary) 
m.x_2 = pyo.Var(m.N, domain = Binary) 
m.x_3 = pyo.Var(m.N, domain = Binary) 
m.x_4 = pyo.Var(m.N, domain = Binary) 
m.x_5 = pyo.Var(m.N, domain = Binary)

# objective: maximize sum of 5 months' weighted rating of selected restaurant
m.obj = Objective(expr = sum(m.x_1[n]*weighted_rating[n-1] 
                            +m.x_2[n]*weighted_rating[n-1] 
                            +m.x_3[n]*weighted_rating[n-1] 
                            +m.x_4[n]*weighted_rating[n-1] 
                            +m.x_5[n]*weighted_rating[n-1] for n in m.N),sense=pyo.maximize)

m.c = ConstraintList()

# Every restuarant cannot be selected for more than one month
for n in range(1, N+1):
    m.c.add(m.x_1[n] + m.x_2[n] + m.x_3[n] + m.x_4[n] + m.x_5[n] <= 1)

# For each month, only one restuarant can be picked
m.c.add(sum(m.x_1[n] for n in m.N) == 1)  
m.c.add(sum(m.x_2[n] for n in m.N) == 1)
m.c.add(sum(m.x_3[n] for n in m.N) == 1)
m.c.add(sum(m.x_4[n] for n in m.N) == 1)
m.c.add(sum(m.x_5[n] for n in m.N) == 1)

# For each month, budget constraint: price range of the restuarant picked cannot exceed the budget price range (mean or min of 4 people?)
m.c.add(sum(m.x_1[n]*price_range[n-1] for n in m.N) <= budget[0])
m.c.add(sum(m.x_2[n]*price_range[n-1] for n in m.N) <= budget[1])
m.c.add(sum(m.x_3[n]*price_range[n-1] for n in m.N) <= budget[2])
m.c.add(sum(m.x_4[n]*price_range[n-1] for n in m.N) <= budget[3])
m.c.add(sum(m.x_5[n]*price_range[n-1] for n in m.N) <= budget[4])

# For each month, distance constraint: the max distance of restuarants selected cannot exceed the minimum distance tolerance of 4 people
m.c.add(sum(m.x_1[n]*max_distance[n-1] for n in m.N) <= distance_tolerance[0])
m.c.add(sum(m.x_2[n]*max_distance[n-1] for n in m.N) <= distance_tolerance[1])
m.c.add(sum(m.x_3[n]*max_distance[n-1] for n in m.N) <= distance_tolerance[2])
m.c.add(sum(m.x_4[n]*max_distance[n-1] for n in m.N) <= distance_tolerance[3])
m.c.add(sum(m.x_5[n]*max_distance[n-1] for n in m.N) <= distance_tolerance[4])



In [59]:
SolverFactory('cbc').solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': -17.39740964, 'Upper bound': -17.39740964, 'Number of objectives': 1, 'Number of constraints': 246, 'Number of variables': 1155, 'Number of binary variables': 1155, 'Number of integer variables': 1155, 'Number of nonzeros': 1155, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.04, 'Wallclock time': 0.06, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.07930326461791992}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [60]:
result = [[i for i in m.N if m.x_1[i]()],
[i for i in m.N if m.x_2[i]()],
[i for i in m.N if m.x_3[i]()],
[i for i in m.N if m.x_4[i]()],
[i for i in m.N if m.x_5[i]()]]

result

[[4], [10], [3], [2], [1]]